#### About
1. Conditional GANs are GANs that allow us to condition the network with additional information such as class labels. It means that during the phase of training, Images are passed to the network along with their actual class labels for it to learn the difference between them.
2. The limitation of generating random samples with a GAN is overcome via a conditional GAN i.e control on output is maintained. For e.g - In Fashion MNIST, CGAN can help output all jacket's image or equivalent customization.
3. In this notebook, We'll implement CGANs on MNIST Hand written digits dataset.
4. The loss function of GANs quoted below
![gan_loss.png](gan_loss.png)
is modified by conditioning class labels as i.e conditional probabilities
![cgan_loss.png](cgan_loss.png)